## Projeto de Análise de Dados referente a Vacinação no Brasil
### Fonte dos dados: https://opendatasus.saude.gov.br/dataset/doses-aplicadas-pelo-programa-de-nacional-de-imunizacoes-pni-2025
### É utilizado os dados do mês de janeiro/2025. 
### Veja que há 3 extensões de arquivos oferecidos no site, sendo que a baixada foi a .csv.

### Objetivo: realizar ETL para que os dados estejam adequados para o desenvolvimento de um dashboard no Looker Studio.
#### Esse script contém: extração dos dados; tratamentos; padronizações; nova coluna; carga para o BigQuery (BQ).

#### **Nota: PNI (Programa Nacional de Imunizações)**

#### Bibliotecas

In [ ]:
import re
import unicodedata
import pandas as pd
import pandas_gbq as pd_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

#### 1) Extração e Entendimento dos dados

In [2]:
# Extração dos dados via .csv disponibilizado no site oficial
df_vacinas = pd.read_csv('../data/vacinacao_jan_2025.csv', sep=';', encoding='latin1')

C:\Users\laras\AppData\Local\Temp\ipykernel_17052\3205199763.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vacinas = pd.read_csv('../data/vacinacao_jan_2025.csv', sep=';', encoding='latin1')


In [3]:
# Informações gerais
df_vacinas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564311 entries, 0 to 2564310
Data columns (total 56 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   co_documento                     object 
 1   co_paciente                      object 
 2   tp_sexo_paciente                 object 
 3   co_raca_cor_paciente             int64  
 4   no_raca_cor_paciente             object 
 5   co_municipio_paciente            float64
 6   co_pais_paciente                 float64
 7   no_municipio_paciente            object 
 8   no_pais_paciente                 object 
 9   sg_uf_paciente                   object 
 10  nu_cep_paciente                  float64
 11  ds_nacionalidade_paciente        object 
 12  no_etnia_indigena_paciente       object 
 13  co_etnia_indigena_paciente       object 
 14  co_cnes_estabelecimento          int64  
 15  no_razao_social_estabelecimento  object 
 16  no_fantasia_estalecimento        object 
 17  co_munic

In [4]:
# (linhas, colunas)
df_vacinas.shape

(2564311, 56)

In [5]:
# Visualizar as primeiras linhas
df_vacinas.head()

,co_documento,co_paciente,tp_sexo_paciente,co_raca_cor_paciente,no_raca_cor_paciente,co_municipio_paciente,co_pais_paciente,no_municipio_paciente,no_pais_paciente,sg_uf_paciente,...,ds_condicao_maternal,co_tipo_estabelecimento,ds_tipo_estabelecimento,co_natureza_estabelecimento,ds_natureza_estabelecimento,nu_idade_paciente,co_condicao_maternal,no_uf_paciente,no_uf_estabelecimento,dt_deletado_rnds
0,01fe40f1-4c4d-4341-b387-572433310245-i0b0,775d8c38b7385c09388cc965b7508af2504f1a5d939adf...,M,1,BRANCA,500270.0,10.0,CAMPO GRANDE,BRASIL,MS,...,NaN,5.0,HOSPITAL GERAL,3.0,ENTIDADES SEM FINS LUCRATIVOS,22,NaN,MATO GROSSO DO SUL,MATO GROSSO DO SUL,NaN
1,66e1f0c0-b8fd-4ebd-9e57-cfa0da9362f9-i0b0,d62f05e93b9ef589242ab65e18ed3775ba606fa473f753...,M,3,PARDA,500790.0,10.0,SIDROLANDIA,BRASIL,MS,...,NaN,5.0,HOSPITAL GERAL,3.0,ENTIDADES SEM FINS LUCRATIVOS,20,NaN,MATO GROSSO DO SUL,MATO GROSSO DO SUL,NaN
2,67fa2566-ad13-4be4-9722-63c6b5ae9bb4-i0b0,e2fa00064c9b87f238a1478cd1ba36bc4efcc2771a65b5...,F,3,PARDA,310620.0,10.0,BELO HORIZONTE,BRASIL,MG,...,NaN,5.0,HOSPITAL GERAL,3.0,ENTIDADES SEM FINS LUCRATIVOS,54,NaN,MINAS GERAIS,MINAS GERAIS,NaN
3,383a21eb-3ada-4091-bfe1-885f0aee6679-i0b0,286bb84fbf4926c1c6b1c2d1bdb40d4a3ed2b2d68b7065...,M,3,PARDA,260860.0,10.0,LAGOA DO OURO,BRASIL,PE,...,NaN,5.0,HOSPITAL GERAL,3.0,ENTIDADES SEM FINS LUCRATIVOS,26,NaN,PERNAMBUCO,MATO GROSSO DO SUL,NaN
4,038bd49b-f432-4de0-9a31-48a443edf78d-i0b0,c49f8af2d2c8512a99a958481888bd0caf2e03437048bf...,F,3,PARDA,330550.0,10.0,SAQUAREMA,BRASIL,RJ,...,NaN,2.0,CENTRO DE SAUDE/UNIDADE BASICA,1.0,ADMINISTRACAO PUBLICA,56,NaN,RIO DE JANEIRO,RIO DE JANEIRO,NaN


In [6]:
# Resumo estatístico
df_vacinas.describe()

,co_raca_cor_paciente,co_municipio_paciente,co_pais_paciente,nu_cep_paciente,co_cnes_estabelecimento,co_municipio_estabelecimento,co_vacina,co_dose_vacina,co_local_aplicacao,co_via_administracao,co_sistema_origem,co_estrategia_vacinacao,co_origem_registro,co_vacina_grupo_atendimento,co_vacina_categoria_atendimento,co_vacina_fabricante,co_tipo_estabelecimento,co_natureza_estabelecimento,nu_idade_paciente,co_condicao_maternal
count,2.564311e+06,2.521549e+06,2.521635e+06,2.371712e+06,2.564311e+06,2.564308e+06,2.564311e+06,2.564311e+06,2.564311e+06,2.564311e+06,2.564311e+06,2.533188e+06,141641.000000,465934.000000,459408.000000,1.155617e+06,2.564308e+06,2.564308e+06,2.564311e+06,216753.000000
mean,2.148897e+01,3.226775e+05,1.000087e+01,4.922276e+04,4.109818e+07,3.235593e+05,4.230498e+01,9.708255e+00,1.478029e+00,3.000081e+03,2.269094e+04,1.346211e+00,48.159234,270.294949,2.761452,6.806208e+03,6.335845e+00,1.063910e+00,1.265547e+01,1.016489
std,3.862026e+01,1.021727e+05,2.289369e-01,2.803264e+04,3.469240e+10,1.014803e+05,2.892807e+01,1.321678e+01,6.051599e+00,5.493052e+03,8.140565e+03,1.439195e+00,48.965578,256.308764,5.203991,1.295616e+04,1.574460e+01,2.829583e-01,1.913904e+01,0.128679
min,1.000000e+00,1.100010e+05,1.000000e+01,0.000000e+00,4.300000e+01,1.100010e+05,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.634100e+04,0.000000e+00,1.000000,0.000000,1.000000,1.420000e+02,1.000000e+00,1.000000e+00,0.000000e+00,1.000000
25%,1.000000e+00,2.606400e+05,1.000000e+01,2.272300e+04,2.211890e+06,2.608450e+05,2.200000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.860200e+04,1.000000e+00,1.000000,210.000000,2.000000,1.490000e+02,2.000000e+00,1.000000e+00,0.000000e+00,1.000000
50%,3.000000e+00,3.304140e+05,1.000000e+01,5.535500e+04,2.631962e+06,3.304550e+05,3.400000e+01,3.000000e+00,0.000000e+00,0.000000e+00,1.860200e+04,1.000000e+00,1.000000,210.000000,2.000000,1.520000e+02,2.000000e+00,1.000000e+00,3.000000e+00,1.000000
75%,4.000000e+00,3.550300e+05,1.000000e+01,7.158700e+04,4.886925e+06,3.550300e+05,5.600000e+01,9.000000e+00,1.000000e+00,1.089000e+04,2.429400e+04,1.000000e+00,99.000000,210.000000,2.000000,1.650000e+02,2.000000e+00,1.000000e+00,1.600000e+01,1.000000
max,9.900000e+01,9.999990e+05,1.710000e+02,9.999900e+04,3.207445e+13,5.300100e+05,1.120000e+02,8.200000e+01,9.900000e+01,7.790380e+05,4.850300e+04,1.400000e+01,99.000000,2001.000000,446.000000,4.349900e+04,8.500000e+01,4.000000e+00,1.250000e+02,3.000000


In [7]:
# Valores nulos por coluna
df_vacinas.isnull().sum()

co_documento                             0
co_paciente                              0
tp_sexo_paciente                         0
co_raca_cor_paciente                     0
no_raca_cor_paciente                     0
co_municipio_paciente                42762
co_pais_paciente                     42676
no_municipio_paciente                42678
no_pais_paciente                     42676
sg_uf_paciente                       43103
nu_cep_paciente                     192599
ds_nacionalidade_paciente              295
no_etnia_indigena_paciente         2554030
co_etnia_indigena_paciente         2554030
co_cnes_estabelecimento                  0
no_razao_social_estabelecimento          3
no_fantasia_estalecimento                3
co_municipio_estabelecimento             3
no_municipio_estabelecimento             3
sg_uf_estabelecimento                    3
co_troca_documento                 2535265
co_vacina                                0
sg_vacina                                0
dt_vacina  

#### 2) Tratamento dos dados

##### 2.1) Analisando colunas com mais de 2.000.000 de registros nulos

In [ ]:
# Analisando as colunas que contém, no mínimo, 2.000.000 de registros nulos
col_com_nulos_limite = df_vacinas.columns[df_vacinas.isnull().sum() > 2000000]
df_vacinas_com_nulos = df_vacinas[col_com_nulos_limite]
df_vacinas_com_nulos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564311 entries, 0 to 2564310
Data columns (total 12 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   no_etnia_indigena_paciente       object 
 1   co_etnia_indigena_paciente       object 
 2   co_troca_documento               object 
 3   co_origem_registro               float64
 4   ds_origem_registro               object 
 5   co_vacina_grupo_atendimento      float64
 6   ds_vacina_grupo_atendimento      object 
 7   co_vacina_categoria_atendimento  float64
 8   ds_vacina_categoria_atendimento  object 
 9   ds_condicao_maternal             object 
 10  co_condicao_maternal             float64
 11  dt_deletado_rnds                 object 
dtypes: float64(4), object(8)
memory usage: 234.8+ MB


In [9]:
# Valores não nulos
df_vacinas_com_nulos.count()

no_etnia_indigena_paciente          10281
co_etnia_indigena_paciente          10281
co_troca_documento                  29046
co_origem_registro                 141641
ds_origem_registro                  73481
co_vacina_grupo_atendimento        465934
ds_vacina_grupo_atendimento        465934
co_vacina_categoria_atendimento    459408
ds_vacina_categoria_atendimento    459408
ds_condicao_maternal               216753
co_condicao_maternal               216753
dt_deletado_rnds                    22624
dtype: int64

In [10]:
# Remoção de colunas com valores preenchidos até 5% dos dados totais
df_vacinas = df_vacinas.drop(['no_etnia_indigena_paciente', 'co_etnia_indigena_paciente', 
                              'co_troca_documento', 'ds_origem_registro', 'dt_deletado_rnds'], axis=1)
df_vacinas.shape

(2564311, 51)

In [11]:
# Remoção das demais colunas com base em critérios pré-definidos
# Colunas de códigos que são melhores representados por suas colunas descritivas (iniciam com ds_mesmo_nome)
# Estas são númericas
# co_origem_registro, co_vacina_grupo_atendimento, co_condicao_maternal, co_vacina_categoria_atendimento

# Restaram as colunas: ds_vacina_grupo_atendimento, ds_vacina_categoria_atendimento, ds_condicao_maternal
df_vacinas = df_vacinas.drop(['co_origem_registro', 'co_vacina_grupo_atendimento', 'co_condicao_maternal', 
                              'co_vacina_categoria_atendimento'], axis=1)
df_vacinas.shape

(2564311, 47)

##### 2.2) Analisando colunas que relacionam código com a descrição textual 
- Nomenclatura código: co_nome_variavel
- Nomenclatura textual: ds_nome_variavel

In [ ]:
# Lista vazia para inserir o nome das colunas a serem removidas
col_a_serem_excluidas = []

# Listagem das colunas que iniciam com co_, ds_ ou no_
# co_: código
# ds_: descrição
# no_: nome
colunas_co_ds = [col for col in df_vacinas.columns if re.match(r'^(co_|ds_|no_)', col)]

# Criar um df apenas com as colunas dos padrões
df_co_ds = df_vacinas[colunas_co_ds]
df_co_ds.columns

Index(['co_documento', 'co_paciente', 'co_raca_cor_paciente',
       'no_raca_cor_paciente', 'co_municipio_paciente', 'co_pais_paciente',
       'no_municipio_paciente', 'no_pais_paciente',
       'ds_nacionalidade_paciente', 'co_cnes_estabelecimento',
       'no_razao_social_estabelecimento', 'no_fantasia_estalecimento',
       'co_municipio_estabelecimento', 'no_municipio_estabelecimento',
       'co_vacina', 'co_dose_vacina', 'ds_dose_vacina', 'co_local_aplicacao',
       'ds_local_aplicacao', 'co_via_administracao', 'ds_via_administracao',
       'co_lote_vacina', 'ds_vacina_fabricante', 'co_sistema_origem',
       'ds_sistema_origem', 'co_estrategia_vacinacao',
       'ds_estrategia_vacinacao', 'ds_vacina_grupo_atendimento',
       'ds_vacina_categoria_atendimento', 'co_vacina_fabricante', 'ds_vacina',
       'ds_condicao_maternal', 'co_tipo_estabelecimento',
       'ds_tipo_estabelecimento', 'co_natureza_estabelecimento',
       'ds_natureza_estabelecimento', 'no_uf_paciente',
  

In [ ]:
# Analisando as colunas 'co_documento' e 'co_paciente'
# Colunas importantes para representar a quantidade de pacientes registrados nesse conjunto
# Apesar do conteúdo delas não ser útil, visualmente, para o dashboard, em contagem, elas são!
df_co_ds.iloc[:, [0, 1]].sample(12)

# Há registros nulos?
df_co_ds.iloc[:, [0, 1]].isnull().sum()

# São únicas ou há duplicados?
print(f"Há {df_co_ds['co_documento'].duplicated().sum()} registros duplicados em 'co_documento' e {df_co_ds['co_paciente'].duplicated().sum()} em 'co_paciente'.")

# Mantidas? Sim
# Motivos: são importantes para quantificação de pacientes que tomaram vacina no período determinado
# Verificar se a repetição para a coluna 'co_paciente' é proposital ou é o mesmo paciente tomando vacinas diferentes no período

Há 0 registros duplicados em 'co_documento' e 1149731 em 'co_paciente'.


In [14]:
# Analisando a coluna 'co_raca_cor_paciente'
df_co_ds[['co_raca_cor_paciente', 'no_raca_cor_paciente']].value_counts()

# Mantida? Não, apenas a de descrição
# Motivo das excluídas: o código não agrega tanto valor como a descrição
col_a_serem_excluidas.append('co_raca_cor_paciente')

In [15]:
# Analisando as colunas 'co_municipio_paciente', 'co_pais_paciente', 'no_municipio_paciente', 'no_pais_paciente' e 'ds_nacionalidade_paciente'
df_co_ds[['co_municipio_paciente', 'no_municipio_paciente']].value_counts()
# df_co_ds[['co_municipio_paciente', 'no_municipio_paciente']].sample(12)

df_co_ds[['co_pais_paciente', 'no_pais_paciente']].value_counts()

df_co_ds['ds_nacionalidade_paciente'].value_counts()

# Mantida? Apenas no_municipio_paciente, no_pais_paciente e ds_nacionalidade_paciente
# Motivo das excluídas: o código não agrega tanto valor como a descrição
col_a_serem_excluidas.append('co_municipio_paciente')
col_a_serem_excluidas.append('co_pais_paciente')

In [16]:
# Analisando as colunas 'co_cnes_estabelecimento', 'no_razao_social_estabelecimento', 'no_fantasia_estalecimento',
# 'co_municipio_estabelecimento', 'no_municipio_estabelecimento'
df_co_ds[['co_cnes_estabelecimento', 'no_razao_social_estabelecimento', 'no_fantasia_estalecimento']].sample(23)

df_co_ds[['co_municipio_estabelecimento', 'no_municipio_estabelecimento']].value_counts()

# Mantida? Todas, exceto o co_municipio_estabelecimento
# Motivo das excluídas: o código não agrega tanto valor como a descrição
col_a_serem_excluidas.append('co_municipio_estabelecimento')

In [17]:
# Analisando as colunas 'co_vacina', 'co_dose_vacina', 'ds_dose_vacina', 'co_local_aplicacao', 'ds_local_aplicacao', 'co_via_administracao', 
# 'ds_via_administracao', 'co_lote_vacina', 'ds_vacina_fabricante', 'co_sistema_origem', 'ds_sistema_origem', 'co_estrategia_vacinacao', 
# 'ds_estrategia_vacinacao', 'ds_vacina_grupo_atendimento', 'ds_vacina_categoria_atendimento', 'co_vacina_fabricante', 'ds_vacina'
df_co_ds.iloc[:, [14, 30]].sample(12)

# Mantida? ds_vacina, ds_dose_vacina, ds_local_aplicacao, ds_via_administracao, co_lote_vacina, ds_vacina_fabricante, ds_sistema_origem, ds_estrategia_vacinacao, ds_vacina_grupo_atendimento
# ds_vacina_categoria_atendimento
# Motivo das excluídas: o código não agrega tanto valor como a descrição
colunas_temp = ['co_vacina', 'co_dose_vacina', 'co_local_aplicacao', 'co_via_administracao', 'co_sistema_origem', 'co_estrategia_vacinacao', 'co_vacina_fabricante']
for coluna in colunas_temp:
    col_a_serem_excluidas.append(coluna)

In [18]:
# Analisando as colunas 'ds_condicao_maternal', 'co_tipo_estabelecimento', 'ds_tipo_estabelecimento', 'co_natureza_estabelecimento', 'ds_natureza_estabelecimento', 'no_uf_paciente', 'no_uf_estabelecimento'
df_co_ds.iloc[:, [36, 37]].sample(12)

# Mantida? ds_condicao_maternal, ds_tipo_estabelecimento, ds_natureza_estabelecimento, no_uf_paciente, no_uf_estabelecimento
# Motivo das excluídas: o código não agrega tanto valor como a descrição
col_a_serem_excluidas.append('co_tipo_estabelecimento')
col_a_serem_excluidas.append('co_natureza_estabelecimento')

In [19]:
# Exclusão no df_vacinas das colunas contidas na lista 'col_a_serem_excluidas'
df_vacinas = df_vacinas.drop(col_a_serem_excluidas, axis=1)

##### 2.3) Tratamento e Padronização de campos

In [20]:
# Padronizações gerais
# - campos de string que não sejam códigos ou siglas serão padronizados com o padrão de primeira letra maíuscula seguida de minúsculas (.title())
# - remover espaços em branco, erroneamente (.strip())
# - toda acentuação será removida a fim de evitar problemas
# - campos float que são inteiros

# Campos a serem padronizadas via amostra e dicionário de dados
# tp_sexo_paciente, nu_cep_paciente, ds_nacionalidade_paciente, no_razao_social_estabelecimento, no_municipio_estabelecimento, co_lote_vacina, ds_vacina_fabricante, st_documento, ds_tipo_estabelecimento

# O objetivo é melhor compreender o valor do registro OU padronizar casos
# A depender do campo, valores NaN serão padronizados como 'Sem Informacao'

###### 2.3.1) Padronizações gerais

In [21]:
# 1) Remover espaços em brancos
# 2) Campos de string que não sejam códigos ou siglas padronizados com o padrão de primeira letra maíuscula seguida de minúsculas

campos1 = ['no_raca_cor_paciente', 'no_municipio_paciente', 'no_pais_paciente', 'no_razao_social_estabelecimento', 'no_fantasia_estalecimento', 'no_municipio_estabelecimento', 'ds_local_aplicacao', 
           'ds_via_administracao',  'ds_vacina_fabricante', 'ds_sistema_origem', 'st_documento', 'ds_estrategia_vacinacao', 'ds_vacina_grupo_atendimento', 'ds_vacina_categoria_atendimento', 'ds_vacina', 
           'ds_condicao_maternal', 'ds_tipo_estabelecimento', 'ds_natureza_estabelecimento', 'no_uf_paciente', 'no_uf_estabelecimento']

df_vacinas[campos1] = df_vacinas[campos1].apply(lambda col: col.str.strip().str.title())

In [22]:
# 3) Remover acentuação para facilitar o tratamento dos dados e não gerar problemas de campos com valor igual mas diferenciado por acentuação/não acentuação
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
        return texto
    return texto

campos2 = df_vacinas.select_dtypes(include='object').columns

df_vacinas[campos2] = df_vacinas[campos2].map(normalizar_texto)

In [23]:
# 4) Transformar campos numéricos: float -> inteiro
campos3 = df_vacinas.select_dtypes(include=['float64']).columns

df_vacinas[campos3] = df_vacinas[campos3].fillna(0)
df_vacinas[campos3] = df_vacinas[campos3].astype(int)

###### 2.3.2) Padronizações específicas

In [ ]:
# 1) Campo: tp_sexo_paciente
# M: Masculino
# F: Feminino

df_vacinas['tp_sexo_paciente'] = df_vacinas['tp_sexo_paciente'].map(lambda tp: 'Masculino' if tp == 'M' else 'Feminino')

In [ ]:
# 2) Campo: nu_cep_paciente
# 0: Sem Informacao

df_vacinas['nu_cep_paciente'] = df_vacinas['nu_cep_paciente'].map(lambda num: 'Sem Informacao' if num == 0 else num)

In [26]:
# 3) Campo: ds_nacionalidade_paciente
# B: Brasileiro
# E: Estrangeiro
# N: Nao Informado

df_vacinas['ds_nacionalidade_paciente'] = df_vacinas['ds_nacionalidade_paciente'].map(lambda nac: 'Brasileiro' if nac == 'B' else ('Estrangeiro' if nac == 'E' else 'Nao Informado'))

In [27]:
# 4) Campo: no_razao_social_estabelecimento
# NaN: Sem Informacao

df_vacinas['no_razao_social_estabelecimento'] = df_vacinas['no_razao_social_estabelecimento'].fillna('Sem Informacao')

In [28]:
# 5) Campo: no_municipio_estabelecimento
# NaN: Sem Informacao

df_vacinas['no_municipio_estabelecimento'] = df_vacinas['no_municipio_estabelecimento'].fillna('Sem Informacao')

In [29]:
# 6) Campo: co_lote_vacina
# NaN: Sem Informacao

df_vacinas['co_lote_vacina'] = df_vacinas['co_lote_vacina'].fillna('Sem Informacao')

In [ ]:
# 7) Campo: ds_vacina_fabricante

# Definições de padrões em regex para busca de valores 
padrao_bio_manguinhos = r'[Bb][i1][o0]?[\s\-\/:]?([Mm][aã]ngu[i1]nh?[o0]s?)?[\s\-\/:]'
padrao_butantan = r'butant[nm]?'
padrao_biofarma = r'bio[fp]h?arma'

# Função geral de padronização
def padroniza_fabricante(fab):
    fab = str(fab)
    fab = fab.upper().strip()

    if pd.isna(fab) or pd.isnull(fab) or 'NAN' in fab or 'NONE' in fab or 'INFORMADO' in fab or 'REGISTRO DE' in fab:
        return 'Sem Informacao'
    elif 'CRUZ' in fab:
        if not re.search(padrao_bio_manguinhos, fab, flags=re.IGNORECASE):
            return 'Fundacao Oswaldo Cruz'
    elif 'INDIA' in fab or 'SERUM' in fab:
        return 'Serum Institute Of India Ltd.'
    elif re.search(padrao_butantan, fab, flags=re.IGNORECASE):
        return 'Fundacao Butantan'
    elif re.search(padrao_biofarma, fab, flags=re.IGNORECASE):
        return 'Biofarma'
    elif 'IDT BIOLOGIKA' in fab:
        return 'Idt Biologika Gmbh'
    elif 'TAKEDA' in fab:
        return 'Takeda Pharma Ltda.'
    elif 'PANACEA' in fab:
        return 'Panacea Biotec Ltd.'
    elif fab == 'LABORATORIOS PFIZER LTDA':
        return 'Laboratorios Pfizer Ltda.'
    elif fab == 'FUNED':
        return 'Fundacao Ezequiel Dias'
    elif fab == 'PFIZER MANUFACTURING BELGIUM NV - BELGIVA':
        return 'Pfizer Manufacturing Belgium NV - Belgica'
    else:
        return fab


# Chamada da função de padronização
df_vacinas['ds_vacina_fabricante'] = df_vacinas['ds_vacina_fabricante'].apply(padroniza_fabricante)

# Contagem de registros por grupo de ds_vacina_fabricante
# Valores com menos de 10 mil registros serão renomeados com ds_vacina_fabricante = 'Descricao Invalida'
contagem = df_vacinas.groupby('ds_vacina_fabricante')['ds_vacina_fabricante'].transform('size')
mascara = contagem < 10000
df_vacinas.loc[mascara, 'ds_vacina_fabricante'] = 'Descricao Invalida'

In [31]:
# 8) Campo: st_documento
# Entered-In-Error: Sem Informacao
df_vacinas['st_documento'] = df_vacinas['st_documento'].apply(lambda status: 'Sem Informacao' if status == 'Entered-In-Error' else status)

In [32]:
# 9) Campo: ds_tipo_estabelecimento
df_vacinas['ds_tipo_estabelecimento'] = df_vacinas['ds_tipo_estabelecimento'].apply(lambda tipo: 'UBS' if tipo in ['Centro De Saude/Unidade Basica', 'Posto De Saude'] else tipo)

##### 2.4) Novas colunas

In [33]:
# 1) Campo: faixa_etaria
# 0: 'Menos de 1 ano'
# 1 - 3: 'Bebe'
# 4 - 10: 'Crianca'
# 11 - 18: 'Pre/Adolescente'
# 19 - 25: 'Jovem'
# 26 - 59: 'Adulto'
# +60: 'Senior'

def faixa_etaria(idade):
    if idade == 0:
        return 'Menos de 1 ano'
    elif idade <= 3: 
        return 'Bebe'
    elif idade <= 10:
        return 'Crianca'
    elif idade <= 18:
        return 'Pre/Adolescente'
    elif idade <= 25:
        return 'Jovem'
    elif idade <= 59:
        return 'Adulto'
    else:
        return 'Senior'

# Chamada da função de padronização
df_vacinas['faixa_etaria'] = df_vacinas['nu_idade_paciente'].apply(faixa_etaria)

#### 3) Carga dos dados no BigQuery

In [ ]:
# Informações do projeto e tabela
project_id = 'nome_do_seu_projeto'
dataset_id = 'br_vacination' 
table_id = 'vacinaction_jan25'

# Conexão com o BQ
client = bigquery.Client(project=project_id) 

In [ ]:
# Criação da tabela com o esquema de df_vacinas
def create_table_from_dataframe(project_id, dataset_id, table_id, dataframe):
    client = bigquery.Client(project=project_id)
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    # Inferindo o esquema a partir do DataFrame
    schema = []
    for column_name, column_dtype in dataframe.dtypes.items():
        if pd.api.types.is_integer_dtype(column_dtype):
            field_type = 'INTEGER'
        elif pd.api.types.is_float_dtype(column_dtype):
            field_type = 'FLOAT'
        elif pd.api.types.is_bool_dtype(column_dtype):
            field_type = 'BOOLEAN'
        elif pd.api.types.is_datetime64_any_dtype(column_dtype):
            field_type = 'TIMESTAMP'
        else:
            field_type = 'STRING'

        schema.append(bigquery.SchemaField(name=column_name.upper(), field_type=field_type))

    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table) 

    print(f'Tabela {table.table_id} criada com sucesso.')

create_table_from_dataframe(project_id, dataset_id, table_id, df_vacinas)

Tabela vacinaction_jan25 criada com sucesso.


In [ ]:
# Carga dos dados
pd_gbq.to_gbq(df_vacinas, f'{dataset_id}.{table_id}', project_id, if_exists='append', progress_bar=True)
print('Os dados foram adicionados com sucesso!')
print(f'Foram adicionados {len(df_vacinas)} novas linhas no BQ.')

c:\Users\laras\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas_gbq\schema\pandas_to_bigquery.py:157: UserWarning: Could not determine the type of columns: nu_cep_paciente
  warnings.warn(msg)




Os dados foram adicionados com sucesso!

Foram adicionados 2564311 novas linhas no BQ.
